In [1]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)


In [2]:
sys.path

['',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload',
 '/home/kai/.local/lib/python3.5/site-packages',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions',
 '/home/kai/.ipython',
 '/home/kai/data/shiyi/AlphaBoosting/automl']

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs 

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek']#, 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']

In [7]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list):
    features_to_gen = []
    for function in function_list:
        for i in range(2, len(categorical_features)+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

#features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count,
#                                      fe.reverse_cumulative_count, fe.variance, fe.count_std_over_mean])
features_to_gen = get_features_to_gen([fe.count])

features_to_gen

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['DayofMonth', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayofMonth', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}}]

In [8]:
len(features_to_gen)

4

In [9]:
project_path = '/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project1/' 

In [10]:
from automl_app import logger_config

logger_config.config(project_path+'log_project1.log')

In [11]:
def gs_params_gen(model='lgb'):
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    if model == 'lgb':
        params = {    
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_rounds': 20,
            'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
            'num_leaves': np.random.choice([15,31,61,127]),
            'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
            'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
            'feature_fraction': np.random.randint(4,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':70,
            'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
            'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
            'bagging_seed': seed,
            'scale_pos_weight': 2,
            'metric' : 'auc' 
        }
    elif model == 'nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.005,
            'batch_size': np.random.choice([32, 64, 128, 256, 512, 1024, 2048]),
            "pred_batch_size": 100000,
            'max_ep': 1,
            'patience': 5,
            'cat_emb_outdim': 50,
            'dense_units': 50,
            'num_dense_n_layers': 1,
            'drop_rate': 0.2,
            'combined_dense_n_layers': 1
        }
    return params, seed

In [12]:
gs_params_gen('nn')

({'batch_size': 512,
  'cat_emb_outdim': 50,
  'combined_dense_n_layers': 1,
  'dense_units': 50,
  'drop_rate': 0.2,
  'ep_for_lr': 1,
  'lr_fin': 0.005,
  'lr_init': 0.01,
  'max_ep': 1,
  'nn_seed': 23945683,
  'num_dense_n_layers': 1,
  'patience': 5,
  'pred_batch_size': 100000},
 23945673)

In [13]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, gs_params_gen)

/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/app.py:123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.df = pd.concat([self.train, self.test], ignore_index=True)
2018-06-22 12:33:32 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #55  | Run record file [./output/last_run_record.json] found. Continue from the previous run...
2018-06-22 12:33:32 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #84  | save run record
2018-06-22 12:33:32 | INFO  | automl_app.app.AlphaBoosting  | _save_run_record | #148 | val index is saved at ./temp_data/val_index.pkl
2018-06-22 12:33:32 | INFO  | automl_app.app.AlphaBoosting  | _save_run_record | #150 | run record is saved at ./output/last_run_record.json
2018-06-22 12:33:32 | INFO  | automl_app.app.AlphaBoosting  | _

Train on 10000 samples, validate on 200000 samples
Epoch 1/1


2018-06-22 12:33:38 | DEBUG | automl_libs.nn_libs.LearningRateTracker| _show_lr   | #137 | At epoch end: init lr 0.01000, lr_decay 0.01841, iterations 40, current lr 0.00576
2018-06-22 12:33:38 | DEBUG | automl_libs.grid_search       | _nn_gs     | #166 | [do_preds] is True, generating predictions ...



Epoch 00001: roc_auc_val improved from -inf to 0.70494, saving model to ./output/gs_saved_preds/nn_saved_models/nn_1529685213.hdf5


2018-06-22 12:33:38 | INFO  | automl_libs.grid_search       | _nn_gs     | #172 | NN predictions(1529685213) saved in ./output/gs_saved_preds/.
2018-06-22 12:33:38 | DEBUG | automl_libs.grid_search       | gs         | #51  | ./output/nn_grid_search.csv updated
2018-06-22 12:33:39 | DEBUG | automl_libs.nn_libs           | get_model  | #39  | Col [DayOfWeek]: embed dim: input 8, output 50
2018-06-22 12:33:39 | DEBUG | automl_libs.nn_libs           | get_model  | #39  | Col [DepTime]: embed dim: input 27, output 50
2018-06-22 12:33:39 | DEBUG | automl_libs.nn_libs           | get_model  | #39  | Col [Origin]: embed dim: input 292, output 50
2018-06-22 12:33:39 | DEBUG | automl_libs.nn_libs           | get_model  | #39  | Col [Dest]: embed dim: input 293, output 50
2018-06-22 12:33:39 | DEBUG | automl_libs.nn_libs           | get_model  | #39  | Col [DayofMonth]: embed dim: input 32, output 50
2018-06-22 12:33:39 | DEBUG | automl_libs.nn_libs           | get_model  | #39  | Col [Distance]

Train on 10000 samples, validate on 200000 samples
Epoch 1/1


2018-06-22 12:33:42 | DEBUG | automl_libs.nn_libs.LearningRateTracker| _show_lr   | #137 | At epoch end: init lr 0.01000, lr_decay 0.25992, iterations 5, current lr 0.00435



Epoch 00001: roc_auc_val improved from -inf to 0.61782, saving model to ./output/gs_saved_preds/nn_saved_models/nn_1529685219.hdf5


2018-06-22 12:33:42 | DEBUG | automl_libs.grid_search       | _nn_gs     | #166 | [do_preds] is True, generating predictions ...
2018-06-22 12:33:42 | INFO  | automl_libs.grid_search       | _nn_gs     | #172 | NN predictions(1529685219) saved in ./output/gs_saved_preds/.
2018-06-22 12:33:42 | DEBUG | automl_libs.grid_search       | gs         | #51  | ./output/nn_grid_search.csv updated
